***vis_items_times* function**:
* Input: players_info dictionary
* Displays bar chart showing for each item the number of use by players

In [2]:
def vis_items_interacted_times(players_info):
    items= find_list_item(players_info)
    nbitems={}
    for item in items:
        nbitems[item]={}
        #pprint.pprint(players_info[player]["interactions"])
        for player in players_info.keys():
            nb=0
            for type in players_info[player]["interactions"].keys():
                for i in players_info[player]["interactions"][type].keys():
                    if item==i:
                        nb+=len(players_info[player]["interactions"][type][item])
            nbitems[item][player]=nb   
    # Multiple bar chart showing the items selected for each personn.
    number_of_plots = len(nbitems)
    if number_of_plots>0:
        for item in nbitems.keys():
            plt.bar(range(len(nbitems[item])), 
                      nbitems[item].values(),
                       tick_label=list(nbitems[item].keys()))
            plt.title("Interaction with item"+ item)#,fontsize=50)
            plt.xlabel("Players")
            plt.ylabel("Numbers of interaction")
            plt.xticks(rotation=90)
            plt.show()
    else:
        plt.text(0.5, 5, 
              "There is no data to fill the visualization.",
                fontsize=10,
                ha='center')
        plt.ylim(0,10)
        plt.gca().axes.xaxis.set_ticklabels([])
        plt.gca().axes.yaxis.set_ticklabels([])
        plt.title("Interaction with item")
        plt.xlabel("Players")
        plt.ylabel("Numbers of interaction")
        plt.show()

def vis_items_interacted_times_heatMap(players_info):
    label_items=find_list_item(players_info)
    nbitems=[]
    label_players=list(players_info.keys())
    i=0
    for item in label_items:
        nbitems.insert(i,[])
        #pprint.pprint(players_info[player]["interactions"])
        j=0
        for player in players_info.keys():
            nb=0
            for type in players_info[player]["interactions"].keys():
                for ittem in players_info[player]["interactions"][type].keys():
                    if item==ittem:
                        nb+=len(players_info[player]["interactions"][type][item])
            nbitems[i].insert(j,nb)
            j+=1
        nbitems[i]=list(nbitems[i])
        i+=1
    ## Now use the heatmap function to plot the results
    xsize=len(label_players)
    ysize=len(label_items)
    if xsize>0 and ysize>0:
        fig, ax = plt.subplots(figsize=(xsize*3,ysize*2))
        im = ax.imshow(nbitems, cmap='cool', interpolation='nearest')
        cbar=plt.colorbar(im)
        cbar.ax.tick_params(labelsize=len(label_items))
        # We want to show all ticks...
        ax.set_xticks(np.arange(len(label_players)))
        ax.set_yticks(np.arange(len(label_items)))
        # ... and label them with the respective list entries
        ax.set_xticklabels(label_players, fontsize=len(label_items))
        ax.set_yticklabels(label_items, fontsize=len(label_items))
            # Rotate the tick labels and set their alignment.
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
        font=len(label_items)
    else:
        plt.text(0.5, 5, 
          "There is no data to fill the visualization.",
          fontsize=10,
          ha='center')
        plt.ylim(0,10)
        font=10
        plt.gca().axes.xaxis.set_ticklabels([])
        plt.gca().axes.yaxis.set_ticklabels([])
    plt.xlabel("Players", fontsize=font)
    plt.ylabel("Items", fontsize=font)
    plt.title("Heatmap showing interaction with item by players", fontsize=font*1.5)
    plt.show()


def find_list_item(players_info): 
    # find full list of items
    items=set()
    for player in players_info.keys():
        for type in players_info[player]["interactions"].keys():
            if players_info[player]["interactions"][type] != {}:                
                items = items.union(players_info[player]["interactions"][type].keys())
    items = list(items)
    #print(items)
    return items